# API development for download of Sentinel-2 and Landsat-8 data
### User defined mosaicing of harmonised products
_Robin Kohrs & Konstantin Schellenberg, February 2020, GEO 419_  
_Supervisor: John Truckenbrodt¹, Martin Habermeyer²_ <br>
<sub><sub>
¹ Friedrich-Schiller-University Jena, chair of remote sensing <br>
² Deutsches Luft- und Raumfahrtszentrum, earth observation center
</sub></sub>

## Tutorial
This is the extention for nasa_hls. We developed following new functions:

In order to make the program be more flexible, the goal of this extension is to
1. Download tiles by setting spatial (user defined verctor geometry) and temporal inqueries to the HLS server,
2. Mosaic the HDF4 formatted product per band and acquisition date.

As part of the module report will want to add the following post procession tasks: 
3. Calculate spectral indexes from the downloaded products.
4. Plot the results inline with `ipyleafet` and `folium` on a Open Street Map WMS service.

Tasks 1 and 2 are meant to work in accordance with the `nasa_hls` package and aims to lead to a pull request to the repository; 3 and 4 shall give an overview of to use the new utilities and the HLS product.

User guide to the HLS product:
https://hls.gsfc.nasa.gov/wp-content/uploads/2019/01/HLS.v1.4.UserGuide_draft_ver3.1.pdf

## Download HSL files with user input
First, the module nasa_hls is loaded.

In [2]:
%matplotlib inline

import nasa_hls
import sys
import pandas as pd
import os

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# and for later processing in the notebook

For testing purpose, try downloading the kml file:

In [3]:
nasa_hls.download_kml()

UTM tiles already successfully downloaded to:
 /home/aleko-kon/.nasa_hls/.auxdata/utm.kml 



'/home/aleko-kon/.nasa_hls/.auxdata/utm.kml'

make tiles from the user given:
1. spatial geometry
2. date or time span
3. product type (S30 or L30, or both)

The input features should come in the WGS84 coordinate system.
Dates have to be passes as yyyy-mm-dd strings.
(when REQUEST = GetFeature) The time or time range for which to return the results, in ISO8601 format (year-month-date, for example: 2016-01-01). When a single time is specified the service will return data until the specified time. If a time range is specified the result is based on all scenes between the specified dates conforming to the cloud coverage criteria and stacked based on priority setting - e.g. most recent on top. The time range is written as two time values separated by a slash, followed by a second slash and a period parameter (which must be P1D). Optional, default: none (the last valid image is returned). Examples: "TIME=2016-01-01", "TIME=2016-01-01/2016-02-01/P1D".
https://www.sentinel-hub.com/develop/documentation/api/ogc_api/wfs-request

In [6]:
ds = nasa_hls.make_tiles_dataset(shape="/home/aleko-kon/Dokumente/nasa_hls/data/amazon.shp",
                                products=["S30", "L30"],
                                start_date="2018-12-25")
ds

starting date: 2018-12-25
 
UTM tiles already successfully downloaded to:
 /home/aleko-kon/.nasa_hls/.auxdata/utm.kml 



  0%|          | 0/10 [00:00<?, ?it/s]


getting available datasets . . .


100%|██████████| 10/10 [00:13<00:00,  1.34s/it]


[    product   tile       date                                                url
 130     S30  19LDJ 2018-12-26  https://hls.gsfc.nasa.gov/data/v1.4/S30/2018/1...
 324     S30  19LEK 2018-12-26  https://hls.gsfc.nasa.gov/data/v1.4/S30/2018/1...
 456     S30  19LDH 2018-12-26  https://hls.gsfc.nasa.gov/data/v1.4/S30/2018/1...,
     product   tile       date                                                url
 565     L30  19LDJ 2018-12-27  https://hls.gsfc.nasa.gov/data/v1.4/L30/2018/1...
 592     L30  19LEJ 2018-12-27  https://hls.gsfc.nasa.gov/data/v1.4/L30/2018/1...
 619     L30  19LEK 2018-12-27  https://hls.gsfc.nasa.gov/data/v1.4/L30/2018/1...
 663     L30  19LDH 2018-12-27  https://hls.gsfc.nasa.gov/data/v1.4/L30/2018/1...,
     product   tile       date                                                url
 131     S30  19LDJ 2018-12-28  https://hls.gsfc.nasa.gov/data/v1.4/S30/2018/1...
 196     S30  19LEJ 2018-12-28  https://hls.gsfc.nasa.gov/data/v1.4/S30/2018/1...
 325     S30  

Taking this list, the data sources can be downloaded via `download_tiles`. <br> This function call `download_batch` and other methods internally in order to parse the right URLs for download.

In [7]:
nasa_hls.download_tiles(dstdir=os.path.join(os.path.expanduser('~'), 'Dokumente', 'nasa_hls', 'data', 'hdf'),
                        datasets=ds)

100%|██████████| 3/3 [04:26<00:00, 88.78s/it] 


***
## Mosaicing tiles
To simplify the handling of the downloaded files ... we mosaic the data according to the 
`make_mosaic.py`

In [ ]:
path_in = os.path.join(os.path.expanduser('~'), 'Dokumente', 'nasa_hls', 'data', 'hdf' + os.sep)
path_out =  os.path.join(os.path.expanduser('~'), 'Dokumente', 'nasa_hls', 'data', 'tifs' + os.sep)

In [ ]:
make_mosaic_tif()

The output extent can now be clipped to the actual outline of the input geometry.

## Calculate Indexes